In [1]:
import requests
import config
from bs4 import BeautifulSoup as BS
import mysql.connector
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [2]:
cnx = mysql.connector.connect(
  host=config.hostj,
  user=config.userj,
  passwd=config.pwj,
  database = 'nfl_draft_info')

cursor = cnx.cursor()

In [4]:
df_joined = pd.read_csv('df_joined.csv')
df_joined = df_joined.set_index('id')
df_joined = df_joined.drop(df_joined.columns[0], axis=1)

In [19]:
df_joined.head()

,name,contract_years,total_value,gtd_start,gtd_total,year,position,height,weight,forty,vertical,bench,broad_jump,three_cone,shuttle,pick_number
id,,,,,,,,,,,,,,,,
11,Nelson Agholor,4,9377374,8520507,8520507,2015,WR,72,198,4.42,NaN,12.0,NaN,NaN,NaN,20.0
10,Arik Armstead,4,9842630,9842630,9842630,2015,DE,79,292,5.06,34.0,24.0,117.0,7.57,4.53,17.0
6,Vic Beasley,4,14495228,14495228,14495228,2015,OLB,75,246,4.53,41.0,35.0,130.0,6.91,4.15,8.0
3,Amari Cooper,4,22663116,22663116,22663116,2015,WR,73,211,4.42,33.0,NaN,120.0,6.71,3.98,4.0
9,Melvin Gordon,4,10669773,10669773,10669773,2015,RB,73,215,4.52,35.0,19.0,126.0,7.04,4.07,15.0


In [20]:
df_joined['position'].unique()

array(['WR', 'DE', 'OLB', 'RB', 'OT', 'CB', 'QB', 'FS', 'DT', 'C', 'P',
       'TE', 'SS', 'ILB', 'OG', 'EDGE', 'S', 'FB', 'LB', 'DB', 'OL', 'DL',
       'K'], dtype=object)

In [22]:
position_groups = {'WR': 'BK', 'DE': 'DL', 'OLB': 'LB', 'RB': 'BK', 'OT': 'OL', 'CB': 'BK', 'QB': 'QB',
                  'FS': 'BK', 'DT': 'DL', 'C': 'OL', 'P': 'ST', 'TE': 'BK', 'SS': 'BK', 'ILB': 'LB', 'OG': 'OL',
                  'EDGE': 'LB', 'S': 'BK', 'FB': 'BK', 'LB': 'LB', 'DB': 'BK', 'OL': 'OL', 'DL': 'DL',
                  'K': 'ST'}

In [23]:
df_joined['pos_grp'] = df_joined['position'].map(position_groups)

In [24]:
df_joined.head(50)

,name,contract_years,total_value,gtd_start,gtd_total,year,position,height,weight,forty,vertical,bench,broad_jump,three_cone,shuttle,pick_number,pos_grp
id,,,,,,,,,,,,,,,,,
11,Nelson Agholor,4,9377374,8520507,8520507,2015,WR,72,198,4.42,NaN,12.0,NaN,NaN,NaN,20.0,BK
10,Arik Armstead,4,9842630,9842630,9842630,2015,DE,79,292,5.06,34.0,24.0,117.0,7.57,4.53,17.0,DL
6,Vic Beasley,4,14495228,14495228,14495228,2015,OLB,75,246,4.53,41.0,35.0,130.0,6.91,4.15,8.0,LB
3,Amari Cooper,4,22663116,22663116,22663116,2015,WR,73,211,4.42,33.0,NaN,120.0,6.71,3.98,4.0,BK
9,Melvin Gordon,4,10669773,10669773,10669773,2015,RB,73,215,4.52,35.0,19.0,126.0,7.04,4.07,15.0,BK
13,D.J. Humphries,4,8912112,7261824,7261824,2015,OT,77,307,5.12,31.0,26.0,104.0,7.87,4.64,24.0,OL
14,Byron Jones,4,8601941,6993950,6993950,2015,CB,73,199,4.43,44.5,NaN,147.0,6.78,3.94,27.0,BK
2,Marcus Mariota,4,24213974,24213974,24213974,2015,QB,76,222,4.52,36.0,NaN,121.0,6.87,4.11,2.0,QB
8,Andrus Peat,4,11393496,11393496,11393496,2015,OT,79,313,5.18,31.0,NaN,105.0,8.01,4.62,13.0,OL


In [39]:
cols = df_joined.columns.tolist()

In [40]:
cols = cols[:5] + cols[-1:] + cols[5:-1]

In [42]:
cols.pop(7)

'position'

In [43]:
cols

['name',
 'contract_years',
 'total_value',
 'gtd_start',
 'gtd_total',
 'pos_grp',
 'year',
 'height',
 'weight',
 'forty',
 'vertical',
 'bench',
 'broad_jump',
 'three_cone',
 'shuttle',
 'pick_number']

In [47]:
df_joined = df_joined[cols]

In [48]:
df_joined.head()

,name,contract_years,total_value,gtd_start,gtd_total,pos_grp,year,height,weight,forty,vertical,bench,broad_jump,three_cone,shuttle,pick_number
id,,,,,,,,,,,,,,,,
11,Nelson Agholor,4,9377374,8520507,8520507,BK,2015,72,198,4.42,NaN,12.0,NaN,NaN,NaN,20.0
10,Arik Armstead,4,9842630,9842630,9842630,DL,2015,79,292,5.06,34.0,24.0,117.0,7.57,4.53,17.0
6,Vic Beasley,4,14495228,14495228,14495228,LB,2015,75,246,4.53,41.0,35.0,130.0,6.91,4.15,8.0
3,Amari Cooper,4,22663116,22663116,22663116,BK,2015,73,211,4.42,33.0,NaN,120.0,6.71,3.98,4.0
9,Melvin Gordon,4,10669773,10669773,10669773,BK,2015,73,215,4.52,35.0,19.0,126.0,7.04,4.07,15.0


In [49]:
position_dummy = pd.get_dummies(df_joined['pos_grp'],prefix = 'position', drop_first=True)

In [55]:
position_dummy.head(50)

,position_DL,position_LB,position_OL,position_QB,position_ST
id,,,,,
11,0,0,0,0,0
10,1,0,0,0,0
6,0,1,0,0,0
3,0,0,0,0,0
9,0,0,0,0,0
13,0,0,1,0,0
14,0,0,0,0,0
2,0,0,0,1,0
8,0,0,1,0,0


In [56]:
df_joined[['pos_grp']]

,pos_grp
id,
11,BK
10,DL
6,LB
3,BK
9,BK
...,...
592,LB
624,ST
640,DL


In [57]:
df_with_dummies = pd.concat([df_joined, position_dummy], axis = 1)

In [58]:
df_with_dummies.head()

,name,contract_years,total_value,gtd_start,gtd_total,pos_grp,year,height,weight,forty,...,bench,broad_jump,three_cone,shuttle,pick_number,position_DL,position_LB,position_OL,position_QB,position_ST
id,,,,,,,,,,,,,,,,,,,,,
11,Nelson Agholor,4,9377374,8520507,8520507,BK,2015,72,198,4.42,...,12.0,NaN,NaN,NaN,20.0,0,0,0,0,0
10,Arik Armstead,4,9842630,9842630,9842630,DL,2015,79,292,5.06,...,24.0,117.0,7.57,4.53,17.0,1,0,0,0,0
6,Vic Beasley,4,14495228,14495228,14495228,LB,2015,75,246,4.53,...,35.0,130.0,6.91,4.15,8.0,0,1,0,0,0
3,Amari Cooper,4,22663116,22663116,22663116,BK,2015,73,211,4.42,...,NaN,120.0,6.71,3.98,4.0,0,0,0,0,0
9,Melvin Gordon,4,10669773,10669773,10669773,BK,2015,73,215,4.52,...,19.0,126.0,7.04,4.07,15.0,0,0,0,0,0


In [74]:
val_reg_pos = ols('total_value~forty+bench+shuttle+vertical+position_DL+position_LB+position_OL', data=df_with_dummies).fit()
val_reg_pos.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_value   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.152
Method:                 Least Squares   F-statistic:                     9.416
Date:                Wed, 15 Jan 2020   Prob (F-statistic):           1.24e-10
Time:                        16:50:53   Log-Likelihood:                -5526.2
No. Observations:                 330   AIC:                         1.107e+04
Df Residuals:                     322   BIC:                         1.110e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    3.239e+07    1.2e+07      2.710      0.007    8.87e+06    5.59e+07
forty       -6.273e+06   1.97e+06     -3.185      0.002   -1.01e+07    -2.4e+06
bench        7.691e+04   5.81e+04      1.324      0.187   -3.74e+04    1.91e+05
shuttle     -2.365e+06   1.78e+06     -1.328      0.185   -5.87e+06    1.14e+06
vertical     2.753e+05   9.35e+04      2.944      0.003    9.13e+04    4.59e+05
position_DL  6.152e+06   1.16e+06      5.296      0.000    3.87e+06    8.44e+06
position_LB  1.041e+06   8.52e+05      1.221      0.223   -6.36e+05    2.72e+06
position_OL  6.761e+06   1.39e+06      4.874      0.000    4.03e+06    9.49e+06
==============================================================================
Omnibus:                      173.067   Durbin-Watson:                   2.086
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              939.220
Skew:                           2.213   Prob(JB):                    1.12e-204
Kurtosis:                       9.980   Cond. No.                     1.90e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.9e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [75]:
df_joined['yearly'] = df_joined['total_value']/df_joined['contract_years']

In [76]:
df_joined.head()

,name,contract_years,total_value,gtd_start,gtd_total,pos_grp,year,height,weight,forty,vertical,bench,broad_jump,three_cone,shuttle,pick_number,yearly
id,,,,,,,,,,,,,,,,,
11,Nelson Agholor,4,9377374,8520507,8520507,BK,2015,72,198,4.42,NaN,12.0,NaN,NaN,NaN,20.0,2344343.50
10,Arik Armstead,4,9842630,9842630,9842630,DL,2015,79,292,5.06,34.0,24.0,117.0,7.57,4.53,17.0,2460657.50
6,Vic Beasley,4,14495228,14495228,14495228,LB,2015,75,246,4.53,41.0,35.0,130.0,6.91,4.15,8.0,3623807.00
3,Amari Cooper,4,22663116,22663116,22663116,BK,2015,73,211,4.42,33.0,NaN,120.0,6.71,3.98,4.0,5665779.00
9,Melvin Gordon,4,10669773,10669773,10669773,BK,2015,73,215,4.52,35.0,19.0,126.0,7.04,4.07,15.0,2667443.25


In [77]:
cols = df_joined.columns.tolist()

In [78]:
cols

['name',
 'contract_years',
 'total_value',
 'gtd_start',
 'gtd_total',
 'pos_grp',
 'year',
 'height',
 'weight',
 'forty',
 'vertical',
 'bench',
 'broad_jump',
 'three_cone',
 'shuttle',
 'pick_number',
 'yearly']

In [79]:
cols = cols[:4] + cols[-1:] + cols[4:-1]

In [81]:
df_joined = df_joined[cols]

In [82]:
df_joined.head()

,name,contract_years,total_value,gtd_start,yearly,gtd_total,pos_grp,year,height,weight,forty,vertical,bench,broad_jump,three_cone,shuttle,pick_number
id,,,,,,,,,,,,,,,,,
11,Nelson Agholor,4,9377374,8520507,2344343.50,8520507,BK,2015,72,198,4.42,NaN,12.0,NaN,NaN,NaN,20.0
10,Arik Armstead,4,9842630,9842630,2460657.50,9842630,DL,2015,79,292,5.06,34.0,24.0,117.0,7.57,4.53,17.0
6,Vic Beasley,4,14495228,14495228,3623807.00,14495228,LB,2015,75,246,4.53,41.0,35.0,130.0,6.91,4.15,8.0
3,Amari Cooper,4,22663116,22663116,5665779.00,22663116,BK,2015,73,211,4.42,33.0,NaN,120.0,6.71,3.98,4.0
9,Melvin Gordon,4,10669773,10669773,2667443.25,10669773,BK,2015,73,215,4.52,35.0,19.0,126.0,7.04,4.07,15.0


In [83]:
df_with_dummies['yearly'] = df_with_dummies['total_value']/df_with_dummies['contract_years']

In [84]:
cols = df_with_dummies.columns.tolist()

In [85]:
cols = cols[:5] + cols[-1:] + cols[5:-1]

In [86]:
df_with_dummies = df_with_dummies[cols]

In [87]:
df_with_dummies.head()

,name,contract_years,total_value,gtd_start,gtd_total,yearly,pos_grp,year,height,weight,...,bench,broad_jump,three_cone,shuttle,pick_number,position_DL,position_LB,position_OL,position_QB,position_ST
id,,,,,,,,,,,,,,,,,,,,,
11,Nelson Agholor,4,9377374,8520507,8520507,2344343.50,BK,2015,72,198,...,12.0,NaN,NaN,NaN,20.0,0,0,0,0,0
10,Arik Armstead,4,9842630,9842630,9842630,2460657.50,DL,2015,79,292,...,24.0,117.0,7.57,4.53,17.0,1,0,0,0,0
6,Vic Beasley,4,14495228,14495228,14495228,3623807.00,LB,2015,75,246,...,35.0,130.0,6.91,4.15,8.0,0,1,0,0,0
3,Amari Cooper,4,22663116,22663116,22663116,5665779.00,BK,2015,73,211,...,NaN,120.0,6.71,3.98,4.0,0,0,0,0,0
9,Melvin Gordon,4,10669773,10669773,10669773,2667443.25,BK,2015,73,215,...,19.0,126.0,7.04,4.07,15.0,0,0,0,0,0
